In [1]:
import pandas as pd
import os
import sys
import glob
import numpy as np
from xlrd import *
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split


In [2]:
#READ CEXCEL FILES AND CONVERT THEM TO CSV FILES AND COMBINE THEM TO ONE FILE
root = '/home/mahnaz/Desktop/iCAIRD_new_slide_labels'
excel_files_dir  = os.path.join(root + '/excel_files')
csv_files_dir = os.path.join(root + '/csv_files')

filenames = os.listdir(excel_files_dir)
for filename in filenames:
    #print('processing {}'.format(filename))
    if '.xls' in filename:
        read_file = pd.read_excel( excel_files_dir + '/'+ filename)
        read_file.to_csv (csv_files_dir +'/' +filename.split('.')[0] + '.csv', index = None, header=True, encoding = 'utf-8-sig')
        
all_filenames = os.listdir(csv_files_dir)
#combine all files in the list
all_files = pd.concat([pd.read_csv(csv_files_dir+ '/'+ f) for f in all_filenames ])
print('There are total {} cervical and endometrial files'.format(len(all_files)))

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: wi

There are total 3849 cervical and endometrial files


In [3]:
endometrial = all_files[(all_files.SampleType =='EN') & (all_files.ExcludedFromAnnotation==0)]
print('Total number of endometrial slides after exclusion = {}'.format(len(endometrial)))
endometrial.head()

Total number of endometrial slides after exclusion = 1854


,AnnotationBatch,ImageName,SampleID,Site,Microtome,Thickness,SampleType,StainingSiteID,StainingProtocol,Cat,SubCat,Revised,ExcludedFromAnnotation,ExclusionReason
0,68,IC-EN-02528-01,IC-EN-02528,SG,1,4µm,EN,5,neuro (hand staining protocol),Other/benign,Proliferative,0,0,NaN
1,68,IC-EN-02529-01,IC-EN-02529,RAH,1,3µm,EN,5,neuro (hand staining protocol),Other/benign,Menstrual,0,0,NaN
2,68,IC-EN-02531-01,IC-EN-02531,NG,1,3µm,EN,3,paeds protocol,Other/benign,Menstrual,0,0,NaN
3,68,IC-EN-02532-01,IC-EN-02532,RAH,2,4µm,EN,1,routine H&E,Other/benign,Proliferative,0,0,NaN
4,68,IC-EN-02534-01,IC-EN-02534,NG,1,4µm,EN,5,neuro (hand staining protocol),Other/benign,Hormonal,0,0,NaN


In [4]:
endometrial = endometrial.sort_values(by=['AnnotationBatch','ImageName'])
endometrial['Cate'] = endometrial['Cat'].str.lower()
endometrial['SubCat'] = endometrial['SubCat'].str.lower()


endometrial['SubCat'] = endometrial['SubCat'].str.replace('/', '_')
endometrial['Cat'] = endometrial['Cat'].str.replace('/','_')

endometrial['SubCat'] = endometrial['SubCat'].str.replace('hyperplasia with atypia', 'hyperplaysia')
endometrial['SubCat'] = endometrial['SubCat'].str.replace('innactive_atrophic', 'innactive')
endometrial = endometrial[['ImageName','SampleID', 'SampleType','Microtome','Thickness', 'Cat', 'SubCat', 'StainingSiteID', 'StainingProtocol']]
endometrial['ImageName'] = endometrial['ImageName'] +'.isyntax'
endometrial = endometrial.reset_index(drop=True)
endometrial.head()

,ImageName,SampleID,SampleType,Microtome,Thickness,Cat,SubCat,StainingSiteID,StainingProtocol
0,IC-EN-00001-01.isyntax,IC-EN-00001,EN,2,4µm,Other_benign,menstrual,3,paeds protocol
1,IC-EN-00003-01.isyntax,IC-EN-00003,EN,1,3µm,Other_benign,secretory,3,paeds protocol
2,IC-EN-00004-01.isyntax,IC-EN-00004,EN,1,3µm,Other_benign,secretory,1,routine H&E
3,IC-EN-00007-01.isyntax,IC-EN-00007,EN,2,4µm,Malignant,adenocarcinoma,2,routine H&E
4,IC-EN-00009-01.isyntax,IC-EN-00009,EN,3,3µm,Insufficient,insufficient,5,neuro (hand staining protocol)


In [5]:
print('endometrial Set  Categories Count:\n', endometrial['Cat'].value_counts())
print('\n')
print('endometrial  Set SubCategories Count:\n', endometrial['SubCat'].value_counts())

endometrial Set  Categories Count:
 Other_benign    1187
Malignant        551
Insufficient     116
Name: Cat, dtype: int64


endometrial  Set SubCategories Count:
 adenocarcinoma          319
secretory               250
proliferative           248
innactive               233
hormonal                224
menstrual               223
hyperplaysia            155
insufficient            123
carcinosarcoma           56
sarcoma                  15
other                     7
atypical hyperplasia      1
Name: SubCat, dtype: int64


In [6]:
endometrial_original = pd.read_csv(os.path.join(root + '/iCAIRD_Endometrial_Data.csv'))
endometrial_original = endometrial_original.sort_values(by=['Filename'])
endometrial_original = endometrial_original.rename(columns={'subCategory':'SubCategory'})
endometrial_original = endometrial_original.reset_index(drop=True)
print(len(endometrial_original))
endometrial_original.head()

2947


,Filename,BiobankID,SampleType,Microtome,Thickness,Category,SubCategory,StainingSiteID,StainingProtocol,train/valid/test
0,IC-EN-00001-01.isyntax,IC-EN-00001,EN,2,4µm,other_benign,menstrual,3,paeds protocol,train
1,IC-EN-00003-01.isyntax,IC-EN-00003,EN,1,3µm,other_benign,secretory,3,paeds protocol,train
2,IC-EN-00004-01.isyntax,IC-EN-00004,EN,1,3µm,other_benign,secretory,1,routine H&E,train
3,IC-EN-00005-01.isyntax,IC-EN-00005,EN,1,3µm,other_benign,menstrual,8,muscle (hand staining protocol),test
4,IC-EN-00006-01.isyntax,IC-EN-00006,EN,1,4µm,other_benign,innactive_atrophic,1,routine H&E,test


In [7]:
endometrial = endometrial.rename(columns={'ImageName':'Filename','SampleID':'BiobankID', 'Cat':'Category', 'SubCat':'SubCategory'})
endometrial_original_df = endometrial_original[['Filename','train/valid/test']]
merged_df = pd.merge(endometrial_original_df,endometrial, how ='left')
merged_df = merged_df.dropna()
merged_df = merged_df.reset_index(drop=True)
merged_df = merged_df [['Filename','BiobankID', 'SampleType','Microtome','Thickness', 'Category', 'SubCategory', 'StainingSiteID', 'StainingProtocol','train/valid/test']]

merged_df

,Filename,BiobankID,SampleType,Microtome,Thickness,Category,SubCategory,StainingSiteID,StainingProtocol,train/valid/test
0,IC-EN-00001-01.isyntax,IC-EN-00001,EN,2.0,4µm,Other_benign,menstrual,3.0,paeds protocol,train
1,IC-EN-00003-01.isyntax,IC-EN-00003,EN,1.0,3µm,Other_benign,secretory,3.0,paeds protocol,train
2,IC-EN-00004-01.isyntax,IC-EN-00004,EN,1.0,3µm,Other_benign,secretory,1.0,routine H&E,train
3,IC-EN-00007-01.isyntax,IC-EN-00007,EN,2.0,4µm,Malignant,adenocarcinoma,2.0,routine H&E,train
4,IC-EN-00009-01.isyntax,IC-EN-00009,EN,3.0,3µm,Insufficient,insufficient,5.0,neuro (hand staining protocol),train
...,...,...,...,...,...,...,...,...,...,...
1849,IC-EN-02829-01.isyntax,IC-EN-02829,EN,1.0,3µm,Other_benign,innactive,7.0,muscle (hand staining protocol),train
1850,IC-EN-02833-01.isyntax,IC-EN-02833,EN,1.0,3µm,Malignant,carcinosarcoma,1.0,routine H&E,train
1851,IC-EN-02834-01.isyntax,IC-EN-02834,EN,3.0,4µm,Other_benign,menstrual,7.0,muscle (hand staining protocol),train
1852,IC-EN-02835-01.isyntax,IC-EN-02835,EN,3.0,4µm,Other_benign,menstrual,1.0,routine H&E,train


In [8]:
endometrial_train = merged_df[merged_df['train/valid/test'] == 'train']
endometrial_test = merged_df[merged_df['train/valid/test'] == 'test']

print('Total {} train smaples and {} test samples'.format(len(endometrial_train), len(endometrial_test)))
print('\n\n')
print('Endometrial Train Set before split to train/valid  Main Categories Count:\n', endometrial_train['Category'].value_counts())
print('\n')
print('Endometrial Train Set  before split to train/valid  SubCategories Count:\n', endometrial_train['SubCategory'].value_counts())
print('\n')
print('Endometrial Train Set  before split to train/valid  StainingSiteID Count:\n', endometrial_train['StainingSiteID'].value_counts())


print('\n')
print('\n')

print('Endometrial Test Set Main Categories Count:\n',endometrial_test['Category'].value_counts())
print('\n')
print('Endometrial Tes Set SubCategories Count:\n', endometrial_test['SubCategory'].value_counts())
print('\n')
print('Endometrial Test Set StainingSiteID Count:\n',endometrial_test['StainingSiteID'].value_counts())

Total 1854 train smaples and 0 test samples



Endometrial Train Set before split to train/valid  Main Categories Count:
 Other_benign    1187
Malignant        551
Insufficient     116
Name: Category, dtype: int64


Endometrial Train Set  before split to train/valid  SubCategories Count:
 adenocarcinoma          319
secretory               250
proliferative           248
innactive               233
hormonal                224
menstrual               223
hyperplaysia            155
insufficient            123
carcinosarcoma           56
sarcoma                  15
other                     7
atypical hyperplasia      1
Name: SubCategory, dtype: int64


Endometrial Train Set  before split to train/valid  StainingSiteID Count:
 2.0    326
1.0    320
4.0    316
3.0    305
7.0    296
5.0    291
Name: StainingSiteID, dtype: int64




Endometrial Test Set Main Categories Count:
 Series([], Name: Category, dtype: int64)


Endometrial Tes Set SubCategories Count:
 Series([], Name: SubCategory, 

In [9]:
blood_mucus  = pd.read_csv(os.path.join(root + '/bloodm_training.csv'))
blood_mucus_names = blood_mucus['ImageName']

endometrial_train = endometrial_train[~endometrial_train['Filename'].isin(blood_mucus_names)]
print(len(endometrial_train))

from sklearn.model_selection import train_test_split
train, valid = train_test_split(endometrial_train, test_size=0.33, random_state=0, stratify=endometrial_train[['Category','StainingSiteID']])
valid['train/valid/test'] = 'valid'
valid.head()

1854


,Filename,BiobankID,SampleType,Microtome,Thickness,Category,SubCategory,StainingSiteID,StainingProtocol,train/valid/test
1639,IC-EN-02501-01.isyntax,IC-EN-02501,EN,1.0,3µm,Malignant,hyperplaysia,5.0,neuro (hand staining protocol),valid
864,IC-EN-01070-01.isyntax,IC-EN-01070,EN,1.0,3µm,Malignant,hyperplaysia,7.0,muscle (hand staining protocol),valid
1587,IC-EN-02416-01.isyntax,IC-EN-02416,EN,1.0,3µm,Other_benign,innactive,1.0,routine H&E,valid
1576,IC-EN-02399-01.isyntax,IC-EN-02399,EN,2.0,3µm,Malignant,hyperplaysia,2.0,routine H&E,valid
1100,IC-EN-01532-01.isyntax,IC-EN-01532,EN,2.0,3µm,Malignant,adenocarcinoma,1.0,routine H&E,valid


In [10]:
print('Endometrial Train Set Main Categories Count:\n', train['Category'].value_counts())
print('\n')
print('Endometrial Train Set SubCategories Count:\n', train['SubCategory'].value_counts())
print('\n')
print('Endometrial Train Set StainingSiteID Count:\n', train['StainingSiteID'].value_counts())

print('\n')
print('\n')

print('Endometrial valid Set Main Categories Count:\n', valid['Category'].value_counts())
print('\n')
print('Endometrial valid Set SubCategories Count:\n', valid['SubCategory'].value_counts())
print('\n')
print('Endometrial valid Set StainingSiteID Count:\n', valid['StainingSiteID'].value_counts())


Endometrial Train Set Main Categories Count:
 Other_benign    795
Malignant       369
Insufficient     78
Name: Category, dtype: int64


Endometrial Train Set SubCategories Count:
 adenocarcinoma          225
secretory               178
innactive               168
proliferative           153
menstrual               146
hormonal                145
hyperplaysia             92
insufficient             81
carcinosarcoma           38
sarcoma                   9
other                     6
atypical hyperplasia      1
Name: SubCategory, dtype: int64


Endometrial Train Set StainingSiteID Count:
 2.0    219
1.0    214
4.0    212
3.0    204
7.0    198
5.0    195
Name: StainingSiteID, dtype: int64




Endometrial valid Set Main Categories Count:
 Other_benign    392
Malignant       182
Insufficient     38
Name: Category, dtype: int64


Endometrial valid Set SubCategories Count:
 proliferative     95
adenocarcinoma    94
hormonal          79
menstrual         77
secretory         72
innactive    

In [11]:
df = pd.concat([train,valid])
df = df.sort_values(by=['Filename'])
df = df.reset_index(drop=True)
df

,Filename,BiobankID,SampleType,Microtome,Thickness,Category,SubCategory,StainingSiteID,StainingProtocol,train/valid/test
0,IC-EN-00001-01.isyntax,IC-EN-00001,EN,2.0,4µm,Other_benign,menstrual,3.0,paeds protocol,train
1,IC-EN-00003-01.isyntax,IC-EN-00003,EN,1.0,3µm,Other_benign,secretory,3.0,paeds protocol,train
2,IC-EN-00004-01.isyntax,IC-EN-00004,EN,1.0,3µm,Other_benign,secretory,1.0,routine H&E,valid
3,IC-EN-00007-01.isyntax,IC-EN-00007,EN,2.0,4µm,Malignant,adenocarcinoma,2.0,routine H&E,train
4,IC-EN-00009-01.isyntax,IC-EN-00009,EN,3.0,3µm,Insufficient,insufficient,5.0,neuro (hand staining protocol),valid
...,...,...,...,...,...,...,...,...,...,...
1849,IC-EN-02829-01.isyntax,IC-EN-02829,EN,1.0,3µm,Other_benign,innactive,7.0,muscle (hand staining protocol),train
1850,IC-EN-02833-01.isyntax,IC-EN-02833,EN,1.0,3µm,Malignant,carcinosarcoma,1.0,routine H&E,train
1851,IC-EN-02834-01.isyntax,IC-EN-02834,EN,3.0,4µm,Other_benign,menstrual,7.0,muscle (hand staining protocol),valid
1852,IC-EN-02835-01.isyntax,IC-EN-02835,EN,3.0,4µm,Other_benign,menstrual,1.0,routine H&E,train


In [12]:
train['Filename'].to_csv('train_filenamescsv')